In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
from datetime import date as date

In [10]:
# import data from local files
global_mobility = pd.read_csv('/Users/ljy/Desktop/RawData/google_mobility.csv', header=0, names=[
    'country_code', 'country', 'region_1', 'region_2', 'date', 'retail_recreation', \
    'grocery_pharmacy', 'parks', 'transit', 'workplaces', 'residential'], low_memory=False)
covid19_canada = pd.read_csv('/Users/ljy/Desktop/RawData/covid19_canada.csv', low_memory=False)
covid19_global = pd.read_csv('/Users/ljy/Desktop/RawData/covid19_global.csv', low_memory=False)
weather_ontario = pd.read_csv('/Users/ljy/Desktop/RawData/weather_on.csv', low_memory=False)

In [11]:
global_mobility.head()

,country_code,country,region_1,region_2,date,retail_recreation,grocery_pharmacy,parks,transit,workplaces,residential
0,AE,United Arab Emirates,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0


# Create Dataframe for Canada

In [12]:
# create dataframe for Canada 
canada_mobility = global_mobility.loc[global_mobility.country_code == 'CA']

# drop 'region_2' because all values are NaN
canada_mobility = canada_mobility.drop('region_2', 1)

# export data 
canada_mobility.to_csv(path_or_buf='/Users/ljy/Desktop/CleanedData/canada_mobility.csv')

In [13]:
canada_mobility.head()

,country_code,country,region_1,date,retail_recreation,grocery_pharmacy,parks,transit,workplaces,residential
16799,CA,Canada,NaN,2020-02-15,4.0,2.0,10.0,3.0,1.0,0.0
16800,CA,Canada,NaN,2020-02-16,13.0,8.0,41.0,4.0,0.0,-2.0
16801,CA,Canada,NaN,2020-02-17,-12.0,-15.0,63.0,-28.0,-52.0,11.0
16802,CA,Canada,NaN,2020-02-18,-1.0,4.0,6.0,-1.0,-1.0,1.0
16803,CA,Canada,NaN,2020-02-19,1.0,1.0,9.0,0.0,0.0,0.0


# Create dataframe for provinces in Canada

In [14]:
# drop 'Prince Edward Island' since there are many missing values
non_provinces = [np.NaN, 'Northwest Territories', 'Nunavut', 'Yukon', 'Prince Edward Island']
non_provinces_bool = ~canada_mobility.region_1.isin(non_provinces) 
canada_provinces_mobility = canada_mobility[non_provinces_bool]

# drop rows that contain null values in 'parks' and 'transit' columns since they occupy less than 1% of the records
# filling null values with mean/median may not be a good idea 
# since we will be using insights from future data which will not be available at the time
canada_provinces_mobility.parks.isna().sum() / len(canada_provinces_mobility.parks)
canada_provinces_mobility.transit.isna().sum() / len(canada_provinces_mobility.transit)
canada_provinces_mobility = canada_provinces_mobility.dropna(subset=['parks', 'transit'])

# export data
canada_provinces_mobility.to_csv(path_or_buf='/Users/ljy/Desktop/CleanedData/canada_provinces_mobility.csv')

In [15]:
canada_provinces_mobility.tail()

,country_code,country,region_1,date,retail_recreation,grocery_pharmacy,parks,transit,workplaces,residential
18094,CA,Canada,Saskatchewan,2020-05-21,-12.0,20.0,80.0,-38.0,-39.0,13.0
18095,CA,Canada,Saskatchewan,2020-05-22,-21.0,8.0,83.0,-36.0,-37.0,12.0
18096,CA,Canada,Saskatchewan,2020-05-23,-24.0,6.0,68.0,-27.0,-14.0,6.0
18097,CA,Canada,Saskatchewan,2020-05-24,-19.0,3.0,197.0,-24.0,-16.0,3.0
18098,CA,Canada,Saskatchewan,2020-05-25,-13.0,11.0,151.0,-38.0,-39.0,10.0


# Create Google trends park data and Canada provinces park data

In [16]:
# import data
google_trends_park = pd.read_csv('/Users/ljy/Desktop/RawData/google_trends_park.csv')

# convert date to datetime
google_trends_park.date = pd.to_datetime(google_trends_park.date, infer_datetime_format=True)
google_trends_park = google_trends_park[google_trends_park.date <= '2020-05-25']

# export data
google_trends_park.to_csv(path_or_buf='/Users/ljy/Desktop/CleanedData/google_trends_park.csv')

Since Google trends does not provide daily data for the past 12 months and provides daily data for the past 90 days, we used the data of Google search trends for "park" for the past 90 days.

In [17]:
# create dataframe for Canada province park data

# drop unused columns
canada_provinces_park  = canada_provinces_mobility.drop(['country_code', 'country', 'retail_recreation', \
                                                         'grocery_pharmacy', 'transit', 'workplaces', 
                                                         'residential'], axis=1)

# group data by date
canada_provinces_park = canada_provinces_park.groupby('date').mean().reset_index()

# drop date before 2020-03-04 since Google trends data does not contain data before this date
canada_provinces_park = canada_provinces_park[canada_provinces_park.date >= '2020-03-04']

# export data
canada_provinces_park.to_csv(path_or_buf='/Users/ljy/Desktop/CleanedData/canada_provinces_park.csv')

# Standardize data for each category, region_1, date & Create social-distancing score for each date

In [18]:
# import data
canada_provinces_mobility = pd.read_csv('/Users/ljy/Desktop/CleanedData/canada_provinces_mobility.csv')

canada_provinces_mobility.head()

,Unnamed: 0,country_code,country,region_1,date,retail_recreation,grocery_pharmacy,parks,transit,workplaces,residential
0,16900,CA,Canada,Alberta,2020-02-15,7.0,3.0,24.0,7.0,-2.0,-1.0
1,16901,CA,Canada,Alberta,2020-02-16,10.0,-2.0,28.0,3.0,-3.0,-2.0
2,16902,CA,Canada,Alberta,2020-02-17,-7.0,-2.0,10.0,-40.0,-67.0,15.0
3,16903,CA,Canada,Alberta,2020-02-18,-1.0,1.0,-2.0,-9.0,-5.0,2.0
4,16904,CA,Canada,Alberta,2020-02-19,3.0,4.0,21.0,-7.0,-1.0,1.0


In [19]:
# Calculate mean and standard deviation for mobility score by provinces
mean_mobility_provinces = canada_provinces_mobility.groupby('region_1').mean().reset_index()
std_mobility_provinces = canada_provinces_mobility.groupby('region_1').std().reset_index()

In [20]:
# z-score
def standardize_data_z(province, category, i):
    canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category] = \
    (canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category] - \
    mean_mobility_provinces.loc[mean_mobility_provinces.region_1 == province, category][i]).\
    div(std_mobility_provinces.loc[std_mobility_provinces.region_1 == province, category][i]) \


def standardize_data_min_max(province, category):
#     min-max normalization between [-100, 100]
#     canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category] = \
#     (200 * (canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category] - \
#     min(canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category])) /\
#     (max(canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category]) - \
#     min(canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category])) - 100)
#     min-max normalization between [0, 100]
    canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category] = \
    100 * ((canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category] - \
    min(canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category])) / \
    (max(canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category]) - \
    min(canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category])))

In [21]:
std_mobility_provinces.loc[std_mobility_provinces.region_1 == 'Alberta', 'retail_recreation'][0]

21.427732562552603

In [22]:
provinces = ['Alberta', 'British Columbia', 'Manitoba', 'New Brunswick', 'Newfoundland and Labrador', \
             'Nova Scotia', 'Ontario', 'Quebec', 'Saskatchewan']
categories = ['retail_recreation', 'grocery_pharmacy', 'parks', 'transit', 'workplaces', 'residential']

In [23]:
# standardize data by z-score
i = 0
for province in provinces:
    for category in categories:
        standardize_data_z(province, category, i)
    i += 1

In [24]:
# create new feature Social Distancing Score for each row

# canada_provinces_mobility['s_d_score'] = -1 * (canada_provinces_mobility['parks']) + \
#                                                canada_provinces_mobility.iloc[:, -6:-5].sum(axis=1) + \
#                                                canada_provinces_mobility.iloc[:, -3:-2].sum(axis=1) + \
#                                                canada_provinces_mobility['residential']

# canada_provinces_mobility['s_d_score'] = canada_provinces_mobility.iloc[:, -6:-1].sum(axis=1)

canada_provinces_mobility['s_d_score'] = -1 * (canada_provinces_mobility.iloc[:, -6:-2].sum(axis=1)) + \
canada_provinces_mobility['residential']

In [25]:
# standardize data by min_max
i = 0
for province in provinces:
    for category in categories:
        standardize_data_min_max(province, 's_d_score')
        standardize_data_min_max(province, category)
    i += 1

In [26]:
#canada_provinces_mobility = canada_provinces_mobility.drop(['country_code', 'country', 'Unnamed: 0'], axis=1)

# export data
canada_provinces_mobility.to_csv(path_or_buf='/Users/ljy/Desktop/CleanedData/standardized_mobility_data.csv')

# Create social distancing score for each province

In [43]:
# import data
mobility_data = pd.read_csv('/Users/ljy/Desktop/CleanedData/standardized_mobility_data.csv', parse_dates=['date'], \
                           index_col='Unnamed: 0')

# drop column
mobility_data = mobility_data.drop(['Unnamed: 0.1'], axis=1)
mobility_data

,country_code,country,region_1,date,retail_recreation,grocery_pharmacy,parks,transit,workplaces,residential,s_d_score
0,CA,Canada,Alberta,2020-02-15,95.890411,60.563380,37.688442,100.000000,90.476190,2.941176,2.512721
1,CA,Canada,Alberta,2020-02-16,100.000000,53.521127,39.698492,95.238095,89.285714,0.000000,3.848204
2,CA,Canada,Alberta,2020-02-17,76.712329,53.521127,30.653266,44.047619,13.095238,50.000000,39.416651
3,CA,Canada,Alberta,2020-02-18,84.931507,57.746479,24.623116,80.952381,86.904762,11.764706,18.867595
4,CA,Canada,Alberta,2020-02-19,90.410959,61.971831,36.180905,83.333333,91.666667,8.823529,9.391373
...,...,...,...,...,...,...,...,...,...,...,...
895,CA,Canada,Saskatchewan,2020-05-21,66.666667,81.818182,51.652893,39.130435,46.341463,50.000000,24.381790
896,CA,Canada,Saskatchewan,2020-05-22,54.666667,66.233766,52.892562,42.028986,48.780488,46.666667,31.589697
897,CA,Canada,Saskatchewan,2020-05-23,50.666667,63.636364,46.694215,55.072464,76.829268,26.666667,27.251964
898,CA,Canada,Saskatchewan,2020-05-24,57.333333,59.740260,100.000000,59.420290,74.390244,16.666667,3.865940


In [44]:
mobility_data = mobility_data.groupby('region_1').mean().reset_index()
mobility_data.sort_values(by=['s_d_score'], axis=0, ascending=False)

,region_1,retail_recreation,grocery_pharmacy,parks,transit,workplaces,residential,s_d_score
5,Nova Scotia,40.390071,46.330935,19.816842,36.567797,47.271845,46.850000,54.441147
7,Quebec,48.888664,57.434910,33.172698,36.833790,49.097263,48.599717,51.190799
0,Alberta,47.741760,51.066797,26.170456,39.462518,54.832626,43.011066,48.758080
6,Ontario,48.262275,52.184576,31.873808,39.227292,50.547160,44.267848,47.899812
1,British Columbia,42.505501,41.526761,36.227812,35.939308,52.558750,48.958689,47.621814
8,Saskatchewan,49.504950,52.809567,30.488503,44.554455,58.210577,40.990099,44.244466
3,New Brunswick,53.307749,62.801791,30.116959,48.503236,53.611866,39.903846,42.924744
4,Newfoundland and Labrador,52.400389,53.128689,26.324238,43.799058,41.094620,41.296519,39.834350
2,Manitoba,58.290657,57.102056,27.572524,40.617633,60.373790,37.991513,38.410132


In [51]:

    
# Rename columns
mobility_data = mobility_data.rename(columns={'region_1': 'Province', 's_d_score': 'Social Distancing Score', \
                             'retail_recreation': 'Retail & Recreation', \
                             'grocery_pharmacy': 'Grocery & Pharmacy', 'parks': 'Parks', \
                             'transit': 'Transit', 'workplaces': 'Workplaces',
                             'residential': 'Residential'})

# categories = ['Retail & Recreation', 'Grocery & Pharmacy', 'Parks', 'Workplaces', 'Transit', 'Residential']

# export data
mobility_data.to_csv(path_or_buf='/Users/ljy/Desktop/CleanedData/social_distancing_score_province.csv')

In [52]:
mobility_data.sort_values('Social Distancing Score')

,Province,Retail & Recreation,Grocery & Pharmacy,Parks,Transit,Workplaces,Residential,Social Distancing Score
2,Manitoba,58.290657,57.102056,27.572524,40.617633,60.373790,37.991513,38.410132
4,Newfoundland and Labrador,52.400389,53.128689,26.324238,43.799058,41.094620,41.296519,39.834350
3,New Brunswick,53.307749,62.801791,30.116959,48.503236,53.611866,39.903846,42.924744
8,Saskatchewan,49.504950,52.809567,30.488503,44.554455,58.210577,40.990099,44.244466
1,British Columbia,42.505501,41.526761,36.227812,35.939308,52.558750,48.958689,47.621814
6,Ontario,48.262275,52.184576,31.873808,39.227292,50.547160,44.267848,47.899812
0,Alberta,47.741760,51.066797,26.170456,39.462518,54.832626,43.011066,48.758080
7,Quebec,48.888664,57.434910,33.172698,36.833790,49.097263,48.599717,51.190799
5,Nova Scotia,40.390071,46.330935,19.816842,36.567797,47.271845,46.850000,54.441147


# Create dataset for modeling

In [53]:
# import data
modeling_data = pd.read_csv('/Users/ljy/Desktop/CleanedData/standardized_mobility_data.csv', parse_dates=['date'])

# drop unnecessary columns
modeling_data.drop(['Unnamed: 0', 'retail_recreation', 'grocery_pharmacy', 'parks', 'transit', 'workplaces', \
                    'residential'], axis=1, inplace=True)

# rename columns
modeling_data = modeling_data.rename(columns={'region_1': 'Province', 'date': 'Date', \
                                              's_d_score': 'Score'})

# convert float to int
modeling_data['Score'] = modeling_data['Score'].astype(int)

In [57]:
# create data for the daily increase in the number of cases in Canada

# import data for the number cases in Canada
covid19_canada = pd.read_csv('/Users/ljy/Desktop/RawData/covid19_canada.csv')

# format date
covid19_canada.date = pd.to_datetime(covid19_canada.date, format='%d-%m-%Y')

# filter rows
covid19_canada = covid19_canada.loc[covid19_canada.pruid == 1]
covid19_canada = covid19_canada.loc[covid19_canada.date >= '2020-02-05']

# select columns
covid19_canada = covid19_canada[['date', 'numconf']]

# create new feature - number of increase compare to the day before
covid19_canada['New cases'] = covid19_canada.numconf - covid19_canada.numconf.shift(1, fill_value = 7)

# rename columns
covid19_canada = covid19_canada.rename(columns={'date': 'Date', 'numconf': 'Confirmed Cases'})

In [58]:
# create data for the daily increase in the number of cases in Ontario

# import data
covid19_ontario = pd.read_csv('/Users/ljy/Desktop/RawData/covid19_canada.csv')

# format date
covid19_ontario.date = pd.to_datetime(covid19_ontario.date, format='%d-%m-%Y')

# filter rows
covid19_ontario = covid19_ontario.loc[covid19_ontario.pruid == 35]
covid19_ontario = covid19_ontario.loc[covid19_ontario.date >= '2020-02-05']

# select columns
covid19_ontario = covid19_ontario[['date', 'numconf']]

# create new feature - number of increase compare to the day before
covid19_ontario['New cases'] = covid19_ontario.numconf - covid19_ontario.numconf.shift(1, fill_value = 3)

# rename columns
covid19_ontario = covid19_ontario.rename(columns={'date': 'Date', 'numconf': 'Confirmed Cases (Ontario)', \
                                                  'New cases': 'New Cases (Ontario)'})

In [59]:
# create data for the daily increase in the number of cases around the world

# import data
covid19_globe = pd.read_csv('/Users/ljy/Desktop/RawData/covid19_global.csv')

# format date
covid19_globe.Date_reported = covid19_globe.Date_reported.str.slice(stop=10)

# rename columns
covid19_globe = covid19_globe.rename(columns={'Date_reported': 'Date', 'New_cases': 'New Cases (World)', \
                                              'New_deaths': 'New Deaths (World)', 'Cumulative_cases': 'Cumulative Cases (World)', \
                                              'Cumulative_deaths': 'Cumulative Deaths (World)'})

# group by date
covid19_globe = covid19_globe.groupby('Date').sum().reset_index()

# filter rows
covid19_globe = covid19_globe.loc[covid19_globe.Date >= '2020-02-15']

# format date
covid19_globe.Date = pd.to_datetime(covid19_globe.Date, format='%Y-%m-%d')

In [60]:
# clean data for daily temperature and precipitation in Canada

# import data
weather_ontario = pd.read_csv('/Users/ljy/Desktop/RawData/weather_on.csv')

# rename columns
weather_ontario = weather_ontario.rename(columns={'Date/Time': 'Date', 'Mean Temp (°C)': 'Mean Temperature', \
                                                 'Total Precip (mm)': 'Total Precipitation'})

# # select relevant columns
weather_ontario = weather_ontario[['Date', 'Mean Temperature', 'Total Precipitation']]

# # filter rows
weather_ontario = weather_ontario.loc[weather_ontario.Date >= '2020-02-15']
weather_ontario = weather_ontario.loc[weather_ontario.Date <= '2020-06-02']
# weather_ontario = weather_ontario.loc[]


# format date
weather_ontario.Date = pd.to_datetime(weather_ontario.Date, format='%Y-%m-%d')

In [61]:
weather_ontario.isnull().sum()

Date                   0
Mean Temperature       0
Total Precipitation    0
dtype: int64

In [62]:
def letter_score(score):
#     if score >= 70:
#         return 'A'
#     elif score >= 60:
#         return 'B'
#     elif score >= 50:
#         return 'C'
#     elif score >= 40:
#         return 'D'
#     elif score >= 30:
#         return 'E'
#     else:
#         return 'F'
    if score >= 60: return 'A'
    elif score >= 40: return 'B'
    elif score >= 20: return 'C'
    else: return 'D'

In [63]:
# filter rows for Ontario
modeling_data = modeling_data.loc[modeling_data.Province == 'Ontario']

# merge modeling_data with covid19_ontario
modeling_data = modeling_data.merge(covid19_ontario, how='inner', on='Date')

# merge modeling_data with covid19_globe
modeling_data = modeling_data.merge(covid19_globe, how='inner', on='Date', suffixes=('_ontario', '_globe'))

# merge modeling_data with weather_data
modeling_data = modeling_data.merge(weather_ontario, how='inner', on='Date')

# create new variables
modeling_data['Days Since First Case'] = (modeling_data.Date - datetime.datetime.strptime('2020-01-27', '%Y-%m-%d')).dt.days
modeling_data['Letter Score'] = modeling_data.Score.apply(letter_score)

# drop column
modeling_data = modeling_data.drop(['Province'], axis=1)

# export data
modeling_data.to_csv(path_or_buf='/Users/ljy/Desktop/CleanedData/modeling_data.csv')

In [64]:
datetime.datetime.strptime('2020-01-27', '%Y-%m-%d')

datetime.datetime(2020, 1, 27, 0, 0)

# Create dataset for modeling Social Distancing Score in Canada

In [65]:
def prname_to_pruid(name):
    if name == 'Alberta':
        return 48
    elif name == 'British Columbia':
        return 59
    elif name == 'Manitoba':
        return 46
    elif name == 'New Brunswick':
        return 13
    elif name == 'Newfoundland and Labrador':
        return 10
    elif name == 'Nova Scotia':
        return 12
    elif name == 'Ontario':
        return 35
    elif name == 'Quebec':
        return 24
    elif name == 'Saskatchewan':
        return 47

In [66]:
# import data
modeling_data = pd.read_csv('/Users/ljy/Desktop/CleanedData/standardized_mobility_data.csv', parse_dates=['date'])

# drop unnecessary columns
modeling_data.drop(['Unnamed: 0', 'retail_recreation', 'grocery_pharmacy', 'parks', 'transit', 'workplaces', \
                    'residential'], axis=1, inplace=True)

# rename columns
modeling_data = modeling_data.rename(columns={'region_1': 'Province', 'date': 'Date', \
                                              's_d_score': 'Score'})

# convert float to int
modeling_data['Score'] = modeling_data['Score'].astype(int)

# Create new column for pruid
modeling_data['pruid'] = modeling_data.Province.apply(prname_to_pruid)

In [67]:
# create data for daily increase in the number of cases for each province

# import data
covid19_province = pd.read_csv('/Users/ljy/Desktop/RawData/covid19_canada.csv')

# format date
covid19_province.date = pd.to_datetime(covid19_province.date, format='%d-%m-%Y')

# filter rows
covid19_province = covid19_province.loc[~covid19_province.pruid.isin([1, 99, 60, 62, 61])]
covid19_province = covid19_province.loc[covid19_province.date >= '2020-02-05']

# Sort data by provinces
covid19_province = covid19_province.sort_values(by=['pruid','date'])

# Select columns
covid19_province = covid19_province[['pruid', 'prname', 'date', 'numconf', 'numprob', 'numdeaths']]

# create new feature - number of increase compare to the day before
covid19_province['New Cases'] = covid19_province.numconf - covid19_province.numconf.shift(1, fill_value=0)
covid19_province['New Probable'] = covid19_province.numprob - covid19_province.numprob.shift(1, fill_value=0)
covid19_province['New Deaths'] = covid19_province.numdeaths - covid19_province.numdeaths.shift(1, fill_value=0)

# rename columns
covid19_province = covid19_province.rename(columns={'date': 'Date', 'numconf': 'Confirmed', \
                                                    'numprob': 'Probable', 'numdeaths': 'Death', \
                                                    'prname': 'Province'})

# Set negative values to 0
cols = ['New Cases', 'New Probable', 'New Deaths']
for col in cols:
    covid19_province[col] = covid19_province[col].apply(lambda x: 0 if x < 0 else x)

In [68]:
# Station names: 'TORONTO CITY', 'ADDENBROKE ISLAND', 'CAP-TOURMENTE',
#        'ACADIA VALLEY', 'CARTWRIGHT A',
#        'CHETICAMP HIGHLANDS NATIONAL PARK', 'DOAKTOWN AUTO RCS',
#        'CARBERRY CS', 'CORONACH SPC'

def name_to_pruid(name):
    '''Convert province name into pruid'''
    if name == 'TORONTO CITY':
        return 35
    elif name == 'ADDENBROKE ISLAND':
        return 59
    elif name == 'CAP-TOURMENTE':
        return 24
    elif name == 'ACADIA VALLEY':
        return 48
    elif name == 'CARTWRIGHT A':
        return 10
    elif name == 'CHETICAMP HIGHLANDS NATIONAL PARK':
        return 12
    elif name == 'DOAKTOWN AUTO RCS':
        return 13
    elif name == 'CORONACH SPC':
        return 47
    elif name == 'CARBERRY CS':
        return 46

In [70]:
# clean data for daily temperature and precipitation for provinces in Canada

# import data
weather_on = pd.read_csv('/Users/ljy/Desktop/RawData/weather_on.csv')
weather_bc = pd.read_csv('/Users/ljy/Desktop/RawData/weather_bc.csv')
weather_qc = pd.read_csv('/Users/ljy/Desktop/RawData/weather_qc.csv')
weather_ab = pd.read_csv('/Users/ljy/Desktop/RawData/weather_ab.csv')
weather_nl = pd.read_csv('/Users/ljy/Desktop/RawData/weather_nl.csv')
weather_ns = pd.read_csv('/Users/ljy/Desktop/RawData/weather_ns.csv')
weather_nb = pd.read_csv('/Users/ljy/Desktop/RawData/weather_nb.csv')
weather_mb = pd.read_csv('/Users/ljy/Desktop/RawData/weather_mb.csv')
weather_sk = pd.read_csv('/Users/ljy/Desktop/RawData/weather_sk.csv')

# Combine data
all_weather_data = [weather_on, weather_bc, weather_qc, weather_ab, weather_nl, weather_ns, weather_nb, weather_mb, \
                weather_sk]
weather_data = pd.concat(all_weather_data)

# Rename Columns
weather_data = weather_data.rename(columns={'Date/Time': 'Date', 'Mean Temp (°C)': 'Mean Temperature', \
                                            'Total Precip (mm)': 'Total Precipitation'})

weather_data = weather_data.loc[weather_data.Date >= '2020-02-15']
weather_data = weather_data.loc[weather_data.Date <= '2020-05-25']

# Select relevant columns
weather_data = weather_data[['Station Name', 'Date', 'Mean Temperature', 'Total Precipitation']]

# Change Station Name to pruid
weather_data['Station Name'] = weather_data['Station Name'].apply(name_to_pruid)
weather_data = weather_data.rename(columns={'Station Name': 'pruid'})

# Drop rows that contain null values since they only occupy a small percentage of the data
weather_data = weather_data.dropna(axis=0)

# Convert String to datetime
weather_data.Date = pd.to_datetime(weather_data.Date, format='%Y-%m-%d')

In [71]:
# len(weather_data['Mean Temperature'])
weather_data.isnull().sum()

pruid                  0
Date                   0
Mean Temperature       0
Total Precipitation    0
dtype: int64

In [72]:
# create data for the daily increase in the number of cases around the world

# import data
covid19_globe = pd.read_csv('/Users/ljy/Desktop/RawData/covid19_global.csv')

# format date
covid19_globe.Date_reported = covid19_globe.Date_reported.str.slice(stop=10)

# rename columns
covid19_globe = covid19_globe.rename(columns={'Date_reported': 'Date', 'New_cases': 'New Cases (world)', \
                                              'New_deaths': 'New Deaths (world)', 'Cumulative_cases': 'Confirmed (world)', \
                                              'Cumulative_deaths': 'Deaths (world)'})

# group by date
covid19_globe = covid19_globe.groupby('Date').sum().reset_index()

# filter rows
covid19_globe = covid19_globe.loc[covid19_globe.Date >= '2020-02-15']

# format date
covid19_globe.Date = pd.to_datetime(covid19_globe.Date, format='%Y-%m-%d')

In [82]:
# Merge data for Random Forests Classification!
modeling_data_province = modeling_data.merge(covid19_province, on=['pruid', 'Date', 'Province'])
modeling_data_province = modeling_data_province.merge(weather_data, on=['pruid', 'Date'])
modeling_data_province = modeling_data_province.merge(covid19_globe, on='Date')

# Create new features
modeling_data_province['Days Since First Case'] = (modeling_data_province.Date - datetime.datetime.strptime('2020-01-27', '%Y-%m-%d')).dt.days
modeling_data_province['Letter Score'] = modeling_data_province.Score.apply(letter_score)

# Sort
modeling_data_province = modeling_data_province.sort_values(by='Date')

modeling_data_province = modeling_data_province.drop(['Unnamed: 0.1'], axis=1)

# Export data
modeling_data_province.to_csv(path_or_buf='/Users/ljy/Desktop/CleanedData/modeling_data_province.csv')

In [83]:
# modeling_data
# covid19_province
# covid19_globe
# weather_data
# modeling_data_province